In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 1.1 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
import gensim
from tqdm import tqdm
from gensim.utils import simple_preprocess
from nltk import sent_tokenize

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Fake News/news.csv")

In [ ]:
df.dropna(how ='any', axis = 0, inplace = True)

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
df["text"] = df["text"].apply(lambda x: x.lower())

In [ ]:
stopwords = stopwords.words("english")

In [ ]:
df["text"] = df["text"].apply(lambda x: [word for word in x.split() if word not in stopwords]).apply(lambda x: " ".join(x))

In [ ]:
story = []
for i in df["text"]:
  sent = sent_tokenize(i)
  for j in sent:
    story.append(simple_preprocess(j))

In [ ]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
model.build_vocab(story)

In [ ]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(110715490, 113207170)

In [ ]:
df["text"] = df["text"].apply(lambda x: simple_preprocess(x)).apply(lambda x: " ".join(x))

In [ ]:
df.drop(df[(df["text"] == "")].index, inplace = True)

In [ ]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [ ]:
X = []
for doc in tqdm(df['text'].values):
  X.append(document_vector(doc))

100%|██████████| 70791/70791 [56:27<00:00, 20.90it/s]


In [ ]:
X = np.array(X)

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['label'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers

In [ ]:
model = Sequential([
  layers.Dense(32, activation="softmax"),
  layers.Dense(1, activation="sigmoid")
])

In [ ]:
model.compile(
  optimizer = 'adam',
  loss = 'binary_crossentropy',
  metrics = ['accuracy']
)

In [ ]:
model.fit(
  x = X_train,
  y = y_train,
  epochs = 10,
  verbose =1
)

Epoch 1/50
495/495 [==============================] - 2s 2ms/step - loss: 0.6576 - accuracy: 0.6958
Epoch 2/50
495/495 [==============================] - 1s 2ms/step - loss: 0.5104 - accuracy: 0.8533
Epoch 3/50
495/495 [==============================] - 1s 2ms/step - loss: 0.4067 - accuracy: 0.8625
Epoch 4/50
495/495 [==============================] - 1s 2ms/step - loss: 0.3580 - accuracy: 0.8698
Epoch 5/50
495/495 [==============================] - 1s 2ms/step - loss: 0.3339 - accuracy: 0.8734
Epoch 6/50
495/495 [==============================] - 1s 2ms/step - loss: 0.3132 - accuracy: 0.8777
Epoch 7/50
495/495 [==============================] - 1s 2ms/step - loss: 0.2959 - accuracy: 0.8827
Epoch 8/50
495/495 [==============================] - 1s 2ms/step - loss: 0.2833 - accuracy: 0.8857
Epoch 9/50
495/495 [==============================] - 1s 2ms/step - loss: 0.2739 - accuracy: 0.8879
Epoch 10/50
495/495 [==============================] - 1s 2ms/step - loss: 0.2665 - accuracy: 0.8918

In [ ]:
model.evaluate(X_test, y_test)

124/124 [==============================] - 0s 2ms/step - loss: 0.2142 - accuracy: 0.9118


[0.21420150995254517, 0.9118018746376038]